In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import zipfile


zip_ref = zipfile.ZipFile("/content/drive/My Drive/Colab Notebooks/MURA-v1.1.zip", 'r')
zip_ref.extractall("/tmp/SML")
zip_ref.close()

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm
import cv2
from sklearn.preprocessing import LabelEncoder

%matplotlib inline
np.random.seed(1)

In [0]:
df = pd.read_csv("/tmp/SML/MURA-v1.1/train_image_paths.csv",
                 names=["image_path"])

In [0]:
df['Region'] = df["image_path"].map(lambda x: x.split('/')[-4])
df["Region"] = df["Region"].map(lambda x: x.split('XR_')[-1])
df["Label"] = df["image_path"].map(lambda x: x.split('/')[-2])
df["Label"] = df["Label"].map(lambda x: x.split('_')[-1])
df

In [0]:
X = df["image_path"]
y = df["Region"]

In [0]:
X_train_path, X_test_path, y_train, y_test = train_test_split(X, y, test_size = 0.25,stratify=y, random_state = 57)

In [0]:
X_train = list()
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
for path in X_train_path:
    npath = "/tmp/SML/"+path
    image = cv2.imread(npath,0)
    # image = cv2.equalizeHist(image)
    image = clahe.apply(image) 
    X_train.append(cv2.resize(image,(100,100)))

X_train = np.array(X_train)
#X_train = X_train.astype('float32')/255

print('X_train_orig shape:', X_train.shape)

In [0]:
X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1], X_train.shape[2],1))

In [0]:
X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1], X_test.shape[2],1))

In [0]:
X_valid = list()
for path in X_valid_path:
    npath = "/tmp/SML/"+path
    image = cv2.imread(npath,0)
    #image = cv2.equalizeHist(image)
    image = clahe.apply(image)   
    X_valid.append(cv2.resize(image,(100,100)))
X_valid = np.array(X_valid)
#X_test = X_test.astype('float32')/255
print('X_valid_orig shape:', X_valid.shape)

In [0]:
X_train = X_train_orig/255.
X_test = X_test_orig/255

In [0]:
labelencoder_y = LabelEncoder()
y_train = labelencoder_y.fit_transform(y_train)
y_test = labelencoder_y.fit_transform(y_test)

In [0]:
#CONV MODEL 1

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(224,224,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

In [0]:
score = model.evaluate(X_test, y_test, verbose=1)
print ("Loss = " + str(score[0]))
print ("Test Accuracy = " + str(score[1]))

In [0]:
#CONV MODEL 2

model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(5, 5), activation="relu", input_shape=(100,100,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

In [0]:
score = model.evaluate(X_test, y_test, verbose=1)
print ("Loss = " + str(score[0]))
print ("Test Accuracy = " + str(score[1]))

In [0]:
## FFNN model 1
model = Sequential()
model.add(Dense(1024,  activation="relu", input_shape=(10000,)))
model.add(Dropout(0.25))
model.add(Dense(512,  activation="relu"))
model.add(Dropout(0.25))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

In [0]:
score = model.evaluate(X_test, y_test, verbose=1)
print ("Loss = " + str(score[0]))
print ("Test Accuracy = " + str(score[1]))